# A 100x Faster Way To Read Excel Files with Python.

Python users often use Excel files to store and retrieve data, as it is a preferred format for data sharing among business professionals. However, working with Excel files in Python can be slow and inefficient. To overcome this issue, we have compiled a list of five alternative methods for loading data in Python, which can significantly improve data processing speed. By implementing these techniques, we can expect lightning-fast data processing, with an acceleration of up to 3 orders in magnitude. 

Consider a scenario where we aim to import 10 Excel files containing 30000 rows and 31 columns each, resulting in a total size of approximately 80MB. This serves as a typical example of where we need to load transactional data from an ERP system (SAP) into Python for further analysis.

To accomplish this, we will generate simulated data and import the necessary libraries. The usage of pickle and joblib will be elaborated on in subsequent sections of the article.

In [ ]:
import pandas as pd
import numpy as np
from joblib import Parallel, delayed
import time

for file_number in range(10):
    values = np.random.uniform(size=(30000,31))
    pd.DataFrame(values).to_csv(f"Dummy {file_number}.csv")
    pd.DataFrame(values).to_excel(f"Dummy {file_number}.xlsx")
    pd.DataFrame(values).to_pickle(f"Dummy {file_number}.pickle")

## 1.Starting with the First option to Load Data
We will begin by simply loading these files. Initially, we will establish a Pandas Dataframe and subsequently merge each Excel file into it.

In [ ]:
start = time.time()
df = pd.read_excel(“Dummy 0.xlsx”)
for file_number in range(1,10):
 df.append(pd.read_excel(f”Dummy {file_number}.xlsx”))
end = time.time()
print(“Excel:”, end — start)


>> Excel: 53.4

## 2. The second option is to Read .CSV file

Let us consider saving these files in .csv format instead of .xlsx from our ERP/System/SAP.

In [ ]:
start = time.time()
df = pd.read_csv(“Dummy 0.csv”)
for file_number in range(1,10):
 df.append(pd.read_csv(f”Dummy {file_number}.csv”))
end = time.time()
print(“CSV:”, end — start)


>> CSV: 0.632

The loading time for these files is 0.63 seconds, which is almost 10 times quicker!

Python demonstrates a loading speed that is 100 times faster with CSV files compared to Excel files. Let's opt for CSVs.

One drawback is that CSV files tend to be larger than .xlsx files. For instance, in this scenario, .csv files are 9.5MB, while .xlsx files are 6.4MB.

## 3. The third option is to create a Pandas DataFrame

We can enhance the efficiency of our process by altering our approach to creating pandas DataFrames. Rather than adding each file to an already existing DataFrame, we can load each DataFrame separately into a list and subsequently merge the entire list into a single DataFrame.


In [ ]:
start = time.time()
df = []
for file_number in range(10):
 temp = pd.read_csv(f”Dummy {file_number}.csv”)
 df.append(temp)
df = pd.concat(df, ignore_index=True)
end = time.time()
print(“CSV2:”, end — start)


>> CSV2: 0.619

We have decreased the time by a small percentage. This technique may prove to be beneficial when handling larger Dataframes (df >> 100MB). 

## 4. Utilize Joblib for Concurrent CSV Imports
We aim to import 10 files in Python. Rather than importing each file sequentially, consider importing them simultaneously in parallel. This can be achieved effortlessly through the utilization of joblib.

In [ ]:
start = time.time()
def loop(file_number):
 return pd.read_csv(f”Dummy {file_number}.csv”)
df = Parallel(n_jobs=-1, verbose=10)(delayed(loop)(file_number) for file_number in range(10))
df = pd.concat(df, ignore_index=True)
end = time.time()
print(“CSV//:”, end — start)


>> CSV//: 0.386

The performance is almost two times faster compared to the single-core version. Nevertheless, it is important to note that utilizing 8 cores may not necessarily result in an eightfold increase in speed (in this case, I experienced a two-times speed increase by utilizing 8 cores on a Mac Air equipped with the new M1 chip).

## Python Parallelization Made Easy with Joblib
Joblib is a straightforward Python library that enables the execution of a function in parallel. In essence, joblib functions similarly to a list comprehension, with the distinction that each iteration is carried out by a separate thread. Let's take a look at an illustrative example.

In [ ]:
def loop(file_number):
 return pd.read_csv(f”Dummy {file_number}.csv”)
df = Parallel(n_jobs=-1, verbose=10)(delayed(loop)(file_number) for file_number in range(10))
#equivalent to
df = [loop(file_number) for file_number in range(10)]

## 5. The Fifth option is to use pickle
Storing data in pickle files, a specific format utilized by Python, can significantly increase processing speed compared to using .csv files.

###### However, it is important to note that pickle files cannot be easily opened and viewed manually.

In [ ]:
start = time.time()
def loop(file_number):
 return pd.read_pickle(f”Dummy {file_number}.pickle”)
df = Parallel(n_jobs=-1, verbose=10)(delayed(loop)(file_number) for file_number in range(10))
df = pd.concat(df, ignore_index=True)
end = time.time()
print(“Pickle//:”, end — start)


>> Pickle//: 0.072

##### We have managed to reduce the run time by 80%!

- In general, working with pickle files is much faster compared to csv files. 

- It is not possible to directly extract data from a system in pickle files.

Pickles can be used in the following scenarios:

1. If we want to save data from one of our Python processes (without the intention of opening it in Excel) for future use or
2. in another process, it is advisable to save our Dataframes as pickles instead of .csv files.

3. If we need to reload the same file(s) multiple times, it is recommended to save the file as a pickle the first time we open it. This way, we can directly load the pickle version in subsequent instances.

4. It is important to note that pickle files tend to occupy more space on your drive (excluding this specific example).

5. For instance, let's consider a scenario where we work with transactional monthly data. Each month, we load a new set of data. In this case, we can save all the historical data as .pickle files. Whenever we receive a new file, can be loaded once as a .csv file and then saved as a .pickle file for future use.



## In Addition

Let us consider a scenario where you have received Excel files and we are left with no alternative but to load them in their current state. Additionally, we have the option to utilize joblib for parallelization. In comparison to our previous pickle code, the only modification required is to update the loop function.

In [ ]:
start = time.time()
def loop(file_number):
    return pd.read_excel(f"Dummy {file_number}.xlsx")
df = Parallel(n_jobs=-1, verbose=10)(delayed(loop)(file_number) for file_number in range(10))
df = pd.concat(df, ignore_index=True)
end = time.time()
print("Excel//:", end - start)


>> 13.45

We can decrease the loading time by 70%, bringing it down from 50 seconds to just 13 seconds. Additionally, you have the option to utilize this loop for generating pickle files instantly. This will enable you to experience significantly faster loading times when you access these files in the future.

In [ ]:
def loop(file_number):
    temp = pd.read_excel(f"Dummy {file_number}.xlsx")
    temp.to_pickle(f"Dummy {file_number}.pickle")
    return temp

##### - Joblib provides the option to adjust the parallelization backend to reduce unnecessary overhead. Simply specify 
- prefer="threads"
  when utilizing Parallel.

In [ ]:
def loop(file_number):
      return pd.read_pickle(f"Dummy {file_number}.pickle")
  df = Parallel(n_jobs=-1, verbose=0, prefer="threads")(delayed(loop)(file_number) for file_number in range(10))
  df = pd.concat(df, ignore_index=True)